In [20]:
# consolidate all member list data:
import pandas as pd
from datetime import datetime

# need to revise to handle n files
files = ['20220408_study section members.csv', '20220920_study section members.csv', '20230621_study section members.csv', '20240621_study section members.csv']
print(files[0])

df = pd.read_csv(files[0])
for file in files[1:]:
    df = pd.concat([df, pd.read_csv(file)])

today = datetime.today().strftime('%Y%m%d')
df.to_csv(today+'_all reviewers.csv', index=False, header=True)


20220408_study section members.csv


In [14]:
# clean up names in member list data (remove degrees)
import pandas as pd

file = '20240626_all reviewers.csv'

data = pd.read_csv(file)

# strategy: delete everything after the 2nd comma (1st comma used in lastname, firstname)
def find_nth(haystack: str, needle: str, n: int) -> int:
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

# clean up name, save in several forms
data["Name_original"] = data["Name"]
data["Name"] = [name[:find_nth(name, ',', 2)].strip().replace('  ', ' ') for name in data["Name"]]
data["Lastname"] = [name.split(',')[0] for name in data['Name']]
data["First_and_middle"] = [name.split(',')[1].strip() for name in data['Name']]
data['Firstname'] = [name.split(' ')[0] for name in data['First_and_middle']]

# keep only the start date if it's part of a range
data["Date_original"] = data["Date"]
data["Date"] = [meeting[0:8] for meeting in data["Date"]] 

# remove duplicates
data = data.drop_duplicates(subset=["Name","Date","Section Abbreviation"])

# extract zipcode
data['Zipcode'] = [info.split(' ')[-1] for info in data[data.columns[5:11]].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)]

data.to_csv(file[:-4]+'_cleaned.csv', index=False, header=True)


In [27]:
# filter reviewer list by current UT affiliation, using a different NIH Commons List from KW
import pandas as pd
from datetime import datetime

# Import with all names uppercase for easiest matching
UT_PIs = pd.DataFrame(pd.read_csv('UT Austin All People with Commons Role_20240624KW.csv')["Name"].str.upper()) # just grab Name column
all_reviewers = pd.read_csv('20240626_all reviewers_cleaned.csv') # all columns (meeting info)

UT_reviewers = all_reviewers.merge(UT_PIs)
UT_reviewers = UT_reviewers.drop_duplicates(subset=["Name","Date","Section Abbreviation"])

today = datetime.today().strftime('%Y%m%d')
UT_reviewers.to_csv(today+'_UT NIH reviewers_consolidated.csv', index=False, header=True)